# Generating and simulating circuits with OpenFermion-Forest

The QubitOperator datastructure in OpenFermion is the main point of contact between OpenFermion and Forest.  Translation of the QubitOperator to `PauliTerms` and `PauliSums` is the interface that is constructed in the OpenFermion-Forest module. Fortunately, when traversing layers of abstraction in OpenFermion the QubitOperator naturally appears in all types of simulations.  Upon translation into the language of pyquil, connections to the Forest-QVM or an alternative QVM (such as reference-qvm) that understands pyquil `Program` objects can be initialized.   The following demonstration starts with the interface between the QubitOperator data structure and the `PauliTerm` and `PauliSum` data structures of pyquil and then demonstrates how to construct and simulate Hamiltonains starting from OpenFermion. 

In [1]:
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli

The interface contains two methods that mediate the translation of `PauliTerm` and `PauliSums` to `QubitOperators` and vice-versa.  

In [2]:
qubit_op = QubitOperator('X0 Y1 Z2')
pauli_term = qubitop_to_pyquilpauli(qubit_op)
print(pauli_term)

qubit_op_sum = QubitOperator('X1 Y5 Z3', coefficient=8) + QubitOperator('Y4 Z2', coefficient=1.5)
pauli_term_sum = qubitop_to_pyquilpauli(qubit_op_sum)
print(pauli_term_sum)

(1+0j)*X0*Y1*Z2
(1.5+0j)*Z2*Y4 + (8+0j)*X1*Z3*Y5


We can convert back from a `PauliSum` object ot a QubitOperator

In [3]:
reversed_term = pyquilpauli_to_qubitop(pauli_term)
print(reversed_term.isclose(qubit_op))  # should return True
reversed_sum = pyquilpauli_to_qubitop(pauli_term_sum)
print(reversed_sum.isclose(qubit_op_sum))  # should return True`

True
True


Let's generate the hopping terms for the Hubbard model Hamiltonian on four-spatial sites. 

In [4]:
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
from openfermion.utils import hermitian_conjugated

In [5]:
# we'll construct the Hamiltonian terms
hopping_hamiltonian = FermionOperator()

spatial_orbitals = 4
for i in range(spatial_orbitals):
    electron_hop_alpha = FermionOperator(((2 * i, 1), (2 * ((i + 1) % spatial_orbitals), 0)))
    electron_hop_beta = FermionOperator(((2 * i + 1, 1), ((2 * ((i + 1) % spatial_orbitals) + 1), 0)))
    hopping_hamiltonian += electron_hop_alpha + hermitian_conjugated(electron_hop_alpha)
    hopping_hamiltonian += electron_hop_beta + hermitian_conjugated(electron_hop_beta)

We can turn the hopping hamiltonian into `QubitOperator` terms on ```2 * (spatial_orbital)``` qubits using the OpenFermion Jordan-Wigner routine.  openfermion-Forest provides an interface to convert the QubitOperator objects into pyquil objects and generate a Quil program from their exponentiation.  The Quil program was generated by taking each PauliTerm and converting it to a set of gates according to arXiv:1306.3991.  Once the user has data in the pyQuil format, more pyquil tools, such as a Trotterization engine, can be used.

In [6]:
from pyquil.quil import Program
from forestopenfermion import exponentiate
hopping_term_generator = jordan_wigner(hopping_hamiltonian)

pyquil_program = exponentiate(hopping_term_generator)
print(pyquil_program)

RX(pi/2) 2
RX(pi/2) 4
CNOT 2 3
CNOT 3 4
RZ(1.0) 4
CNOT 3 4
CNOT 2 3
RX(-pi/2) 2
RX(-pi/2) 4
H 0
H 6
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 6
RX(pi/2) 4
RX(pi/2) 6
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
RX(-pi/2) 4
RX(-pi/2) 6
RX(pi/2) 3
RX(pi/2) 5
CNOT 3 4
CNOT 4 5
RZ(1.0) 5
CNOT 4 5
CNOT 3 4
RX(-pi/2) 3
RX(-pi/2) 5
H 0
H 2
CNOT 0 1
CNOT 1 2
RZ(1.0) 2
CNOT 1 2
CNOT 0 1
H 0
H 2
H 5
H 7
CNOT 5 6
CNOT 6 7
RZ(1.0) 7
CNOT 6 7
CNOT 5 6
H 5
H 7
H 4
H 6
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
H 4
H 6
H 3
H 5
CNOT 3 4
CNOT 4 5
RZ(1.0) 5
CNOT 4 5
CNOT 3 4
H 3
H 5
H 1
H 3
CNOT 1 2
CNOT 2 3
RZ(1.0) 3
CNOT 2 3
CNOT 1 2
H 1
H 3
RX(pi/2) 0
RX(pi/2) 2
CNOT 0 1
CNOT 1 2
RZ(1.0) 2
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
RX(-pi/2) 2
RX(pi/2) 1
RX(pi/2) 3
CNOT 1 2
CNOT 2 3
RZ(1.0) 3
CNOT 2 3
CNOT 1 2
RX(-pi/2) 1
RX(-pi/2) 3
H 1
H 7
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
CNOT 5 6
CNOT 6 7
RZ(1.0) 7
CNOT 6 7
CNOT 5 6
CNOT 4

The returned value from exponetiate is a pyquil `Program` object.  The object has some nice features such as a dagger function, easy classical control flow construction, and introspection.  The circuit can be simulated with noise or without noise by running on the Forest-QVM or on reference-qvm.  In order to pyquil Programs on the Forest-QVM you'll need to sign up on the [Forest Home Page ](http://www.rigetti.com/forest) for a key.

In [7]:
from pyquil.api import QVMConnection
qvm = QVMConnection()
wf = qvm.wavefunction(pyquil_program)

The resulting `Wavefunction` object from pyQuil contains pretty printing features and the ability to access the wavefunction.

In [8]:
print(wf.amplitudes)

[ 8.94339015e-01+5.55111512e-17j  2.10311262e-17-2.53729266e-16j
  2.41529599e-17+2.63721096e-17j  1.40592886e-32-1.54074396e-32j
  1.57009246e-16+0.00000000e+00j  2.58688741e-17-9.51379687e-17j
 -6.16297582e-33+9.24446373e-33j -1.01226134e-17-7.49108731e-18j
  1.94535048e-17-4.06284798e-17j  3.08148791e-33+1.46370676e-32j
 -4.49028280e-17+1.03787499e-17j -2.03961345e-18+1.99741020e-17j
 -1.23259516e-32-1.07852077e-32j  1.80242859e-17+1.46652026e-17j
 -2.48178197e-18+2.86481249e-18j -8.40020701e-18+1.15303899e-17j
  1.66533454e-16+0.00000000e+00j -4.94229231e-18+3.59600985e-17j
 -7.70371978e-33-9.24446373e-33j -3.24398029e-18+3.50559485e-17j
 -7.85046229e-17+5.55111512e-17j -2.67935033e-17-1.93961841e-17j
 -3.33866608e-17+1.64561210e-17j -6.16297582e-33-1.07852077e-32j
  7.70371978e-33+6.16297582e-33j -4.91690292e-17+4.23067460e-18j
  1.10480308e-17+2.22202434e-17j -4.15476364e-19+1.50596576e-18j
  7.24292819e-17-2.24042609e-17j -1.23259516e-32+3.23556231e-32j
  1.14124700e-17-7.687811

We can also pretty print the wavefunction (on by default) which prints the amplitudes and bitstrings in an easy to read format. 

In [9]:
print(wf)

(0.894339015+0j)|00000000> + (0.3540367091+0j)|00100010> + (-0-0.1934111357j)|10000010> + 0.1934111357j|10100000>
